In [4]:
import numpy as np
import pandas as pd
import sys
sys.path.append("../src/seq_space_lib/")
import sequence_space_lib as seqsp

path='../data/'

## Assemble all datasets

### Vertebrates

In [ ]:
vertinp=['ncbi_ver_alignments_aa_no_dupl_14583_ali_seq_length_alpha_size_var_invar_sites_keff_sites_w_aa_ogid.tsv',
'dnds_dn_ds_means_medians_var_fracfilt_ncbi_vert_dnds_yn00_ynout.list.csv',
'kcoefs_mult_ali_max_k_range_from_data_unique_non_log_0.5_20.coefnr',
'allstats_mean_median_var_ld_ncbi_vert_mean_median_var_dist_unique.csv', 
'pdistm_matrices_unique_num_of_seq_col_150_seqs_or_more.tsv']

vdfr =pd.read_csv('vert_max_dist_pdistm_matrices_unique_levenshtein.csv', header=None)
vdfr=vdfr.rename(columns={0:'OG_id', 1:'max_ld'})
for i in vertinp:
    if i.endswith('.tsv'):
        opendf=pd.read_csv(i, sep='\t')
    else:
        opendf=pd.read_csv(i)
    vdfr=vdfr.merge(opendf, on=['OG_id'])

### Enterobacterales

In [ ]:
#max dist 
emd=pd.read_csv('entero_max_dist_pdistm_matrices_unique_levenshtein.csv', header=None)
emd=emd.rename(columns={0:'OG_id', 1:'max_ld'})
#usage 
eus=pd.read_csv('entero_aa_ali_200seqs_ali_seq_length_alpha_size_var_invar_sites_keff.tsv', sep='\t')
#add number of unique nt sequences
ntu_len=pd.read_csv('nt_fastas_no_duplicated_number_of_seq.csv', header=None, names=['OG_id', 'num_of_seq_nt_unique'], sep=',')
#add number of unique aa sequences
aau_len=pd.read_csv('aa_ali_no_duplicates_num_of_seq.csv', header=None, names=['OG_id', 'num_of_seq_aa_unique'])
#add original number of sequences (non-unique, same for nt and aa)
orig_len=pd.read_csv('nt_aa_ali_orig_num_of_seq.csv', header=None, names=['OG_id', 'num_of_seq_orig'])

edfr=emd.merge(eus.merge(ntu_len.merge(aau_len.merge(orig_len, on='OG_id'), on='OG_id'), on='OG_id'), on='OG_id')

entinp=['allstats_mean_median_var_ld_enterobacterales_unique_levenshtein.csv',
'entero_kcoefs_mult_ali_max_k_range_from_data_non_log_0.5_20.coefnr',
'dnds_dn_ds_means_medians_var_fracfilt_entero_ynout_yn00_2604.list.csv']

for i in entinp:
    opendf=pd.read_csv(i)
    edfr=edfr.merge(opendf, on=['OG_id'])

### Gammaproteobacteria

In [ ]:
#max dist 
gmd=pd.read_csv('gamma_max_dist_pdistm_matrices_unique_levenshtein.csv', header=None)
gmd=gmd.rename(columns={0:'OG_id', 1:'max_ld'})
#usage
gamu=pd.read_csv('all_gammaproteo_eggNOGs_raw_aa_200_seqs_or_more_ali_seq_length_alpha_size_var_invar_sites_keff.tsv', sep='\t')

#add number of unique nt sequences
ntu_len=pd.read_csv('all_gammaproteo_nt_aligned_ungapped_unique_num_of_seq.csv', header=None, names=['OG_id', 'num_of_seq_nt_unique'], sep=',')
#add number of unique aa sequences
aau_len=pd.read_csv('all_gammaproteo_pdistm_num_of_seq.csv', header=None, names=['OG_id', 'num_of_seq_aa_unique'])
#add original number of sequences (non-unique, same for nt and aa)
orig_len=pd.read_csv('all_gammaproteo_raw_aa_num_of_seq.csv', header=None, names=['OG_id', 'num_of_seq_orig'])
#number of sequences in the nt alignment files
orig_len_nt=pd.read_csv('all_gammaproteo_num_of_seq.txt', header=None, names=['OG_id', 'num_of_seq_nt_unique'], sep=',')

gammaall=gmd.merge(gamu.merge(ntu_len.merge(aau_len.merge(orig_len.merge(orig_len_nt, on='OG_id'), on='OG_id'), on='OG_id'), on='OG_id'), on='OG_id')

gammarename={'OD_id':'OG_id',
'LD_mean':'mean_ld',
'LD_median':'median_ld',
'LD_var':'var_ld','mean_seq_len':'mean_seq_len_orig',
'median_seq_len':'median_seq_len_orig'}
gdfr=gammaall.rename(columns=gammarename)

gaminp=['all_gammaproteo_pdistm_num_of_seq_200_seqs_or_more_wo_folder.list_LD.tsv',
'eggnog_gamma_kcoefs_mult_ali_max_k_range_from_data_non_log_0.5_20.coefnr',
'dnds_dn_ds_means_medians_var_fracfilt_gammaproteo_from_nt_ynouts_yn00_2775.list.csv']

for i in gaminp:
    opendf=pd.read_csv(i)
    gdfr=gdfr.merge(opendf, on=['OG_id'])

### COGs

In [ ]:
dim=pd.read_csv('cogs_full_pairali_kcoefs_mult_ali_max_k_range_from_data_non_log_0.5_20.coefnr')
mdist=pd.read_csv('cogs_full_pairali_max_dist_pdistm_matrices_unique_levenshtein.csv', header=None)
mdist=mdist.rename(columns={0:'OG_id', 1:'max_dist'})
ns=pd.read_csv('cogs4877_num_of_seq_200_seqs_or_more.tsv', sep='\t')
ns['OG_id']=[i[:-3] for i in ns['OG_name']]
dd=dim.merge(mdist, on='OG_id', how='inner')
ddn=dd.merge(ns[['num_of_seq', 'OG_id']], on='OG_id', how='inner')
ddn=ddn.rename(columns={'num_of_seq':'num_of_seq_orig'})
numu=pd.read_csv('num_of_seq_unique_cogs2212.csv', header=None)
numu=numu.rename(columns={0:'OG_id', 1:'num_of_seq_unique'})
d2212=ddn.merge(numu, on='OG_id', how='inner')
sl=pd.read_csv('cogs_full_pairali_mean_median_seq_len_orig.csv')
d2212=d2212.merge(sl, on='OG_id')


dim=pd.read_csv('cogs_full_pairali_kcoefs_mult_ali_max_k_range_from_data_non_log_0.5_20_308.coefnr')
mdist=pd.read_csv('cogs_full_pairali_max_dist_pdistm_matrices_unique_levenshtein_308.csv', header=None)
mdist=mdist.rename(columns={0:'OG_id', 1:'max_dist'})
ns=pd.read_csv('cogs4877_num_of_seq_200_seqs_or_more.tsv', sep='\t')
ns['OG_id']=[i[:-3] for i in ns['OG_name']]
dd=dim.merge(mdist, on='OG_id', how='inner')
ddn=dd.merge(ns[['num_of_seq', 'OG_id']], on='OG_id', how='inner')
ddn=ddn.rename(columns={'num_of_seq':'num_of_seq_orig'})
numu=pd.read_csv('num_of_seq_unique_cogs308.csv', header=None)
numu=numu.rename(columns={0:'OG_id', 1:'num_of_seq_unique'})
dall=ddn.merge(numu, on='OG_id', how='inner')
sl=pd.read_csv('cogs_full_pairali_mean_median_seq_len_orig_308.csv')
dall=dall.merge(sl, on='OG_id')
dall2520=pd.concat([dall, d2212])
ldmean=pd.read_csv('allstats_mean_median_var_ld_cogs_full_pairali_unique_levenshtein_2520.csv')
dall2520ld=dall2520.merge(ldmean, on='OG_id')
#rename
cdfr=dall2520ld.rename(columns={'max_dist':'max_ld', 'num_of_seq_unique':'num_of_seq_aa_unique', 'num_of_seq_orig':'num_of_seq_aa_orig'})

### Simulated dataset

In [ ]:
#simulation details
sdfr=pd.read_csv('allstats_simulated_tree_notree_branchscale_dim_dist_dnds_anc_leaves.csv')

#map IDs
simumap=pd.read_csv('/bucket/KondrashovU/seq_space/simulate_evol/data/uniform_f_w_dist/simu_og_pdistm_names_mapping.csv', header=None).rename(columns={0:'OG_id', 1:'SimuID'})

#add topology 
simuni=pd.read_csv('/bucket/KondrashovU/seq_space/topology_all/Simulate_evol/norm_integral_b0_simulate_evol.txt', sep=' ', header=None).rename(columns={0:'SimuID', 1:'norm_integral_b0'})
simui=pd.read_csv('/bucket/KondrashovU/seq_space/topology_all/Simulate_evol/'+'integral_b0_simulate_evol.txt', sep=' ', header=None).rename(columns={0:'SimuID', 1:'integral_b0'})
#merge 
simubetti=simumap.merge(simuni, on='SimuID').merge(simui, on='SimuID')

#add usage
simuu=pd.read_csv('final_all_fastas_5244_ali_seq_length_alpha_size_var_invar_sites_keff.tsv', sep='\t')

#add distance (unique)
simudist=pd.read_csv('allstats_mean_median_var_ld_simu_w_anc_mean_median_var_dist_unique.csv').rename(columns={'OG_id':'SimuID'})
#add max dist column to the main df
vmd =pd.read_csv('simu_max_dist_pdistm_matrices_unique_levenshtein.csv', header=None)
vmd=vmd.rename(columns={0:'SimuID', 1:'max_ld'})

#add dnds from tree 
simu_dndstree=pd.read_csv('dnds_from_tree_list_final_trees_4845_notreefinal_all.tsv',sep='\t')

# add counts of syn subs in simulated trees
dntot=pd.read_csv('list_final_trees_5244_total_nonsyn_subs_count.csv')
ogid=[]
for i in dntot['OG_id']:
    ogid.append('ali'+i[4:])
dntot['OG_id']=ogid

#add dnds from PAML
spaml=pd.read_csv('dnds_dn_ds_means_medians_var_fracfilt_paml_dnds_yn00_simu.list.csv')

#add dim
srange=pd.read_csv('simu_w_anc_unique_dim_winsize0.5_dim_range_from_data_12444.coefnr').rename(columns={'OG_id':'SimuID'})

simualldf=[simubetti,simuu, simudist, vmd, simu_dndstree, dntot, srange, spaml]

#merge all
for i in simualldf:
    sdfr=sdfr.merge(i, on='OG_id')



## Finalize all tables

In [ ]:

# 0. Rename correlation dimensnion
dim_rename={'range_start':'dim_range_start_ur',
'range_end':'dim_range_end_ur',
'dimensionality_data_range_0.5':'dim_ur',
'max_k_R^2':'dim_R^2_ur',
'max_CI':'dim_CI_ur'}

vdfr.rename(columns=dim_rename, inplace=True)
edfr.rename(columns=dim_rename, inplace=True)
gdfr.rename(columns=dim_rename, inplace=True)
sdfr.rename(columns=dim_rename, inplace=True)
cdfr.rename(columns=dim_rename, inplace=True)

# 1. Add dN/dS expectation from usage
for i in [vdfr, edfr, gdfr]:
    i['dnds_from_usage_sites80']=(i['mean_alphabet_size_less02gaps']-1)/19
    i['dnds_from_usage_sites80_cons']=(i['mean_alphabet_size_less02gaps']-1)/7

sdfr['dnds_from_usage']=(sdfr['mean_alphabet_size']-1)/19
sdfr['dnds_from_usage_cons']=(sdfr['mean_alphabet_size']-1)/7


# 2. Add topological sequence dimension

## 3 main natural sequence datasets
for i in [vdfr, edfr, gdfr]:
    seqsp.add_topol_dim_cols(i) # r = median seq len * max dist in the matrix 
    seqsp.add_expected_topol_dim_cols(i, binomial=True) 
    seqsp.add_expected_topol_dim_cols(i, dimfrom='dn/ds', use_col='dnds_mean', binomial=True)
    #add fraction of explored space
    seqsp.fraction_of_explored_space(i)

## simulated families
seqsp.add_topol_dim_cols(sdfr, seqlen_col='ali_len', alpha_col='median_alphabet_size')
seqsp.add_expected_topol_dim_cols(sdfr, binomial=True, use_col='median_alphabet_size', seqlen_col='ali_len')
seqsp.add_expected_topol_dim_cols(sdfr, dimfrom='dn/ds', use_col='dnds_mean', binomial=True, seqlen_col='ali_len')
#add fraction of explored space
seqsp.fraction_of_explored_space(sdfr)

## cogs (no expected dimensionality, since usage and dN/dS are not available)
seqsp.add_topol_dim_cols(cdfr, base_alpha=False, ci=True, seqlen_col='median_seq_len_orig',  prefix='')

edfr=edfr[(edfr['dnds_mean']<=1)&(edfr['num_of_seq_aa_unique']>=200)]
vdfr=vdfr[(vdfr['dnds_mean']<=1)&(vdfr['num_of_seq_aa_unique']>=200)]
gdfr=gdfr[(gdfr['dnds_mean']<=1)&(gdfr['num_of_seq_aa_unique']>=200)]
cdfr=cdfr[(cdfr['num_of_seq_aa_unique']>=200)]

keepcol=['OG_id','SimuID','num_of_seq_aa_orig ','num_of_seq_aa_unique','ali_len','mean_seq_len_orig','median_seq_len_orig','num_invar_sites','num_invar_sites_less02gaps','num_invar_sites_less05gaps','num_invar_sites_no_gaps','num_var_sites','num_var_sites_less02gaps','num_var_sites_less05gaps','num_sites_less02gaps','num_sites_less05gaps','median_frac_gaps_per_site','mean_frac_gaps_per_site','mean_alphabet_size','mean_alphabet_size_less02gaps','mean_alphabet_size_less05gaps','median_alphabet_size','median_alphabet_size_less02gaps','median_alphabet_size_less05gaps','max_ld','mean_ld','median_ld','var_ld','dim_ur','dim_CI_ur','dim_R^2_ur','dim_range_end_ur','dim_range_start_ur','n_seq_from_dim_log10','n_seq_from_dim_log20','n_seq_from_dim_log_alpha','log10_nseq_ci','log20_nseq_ci','n_seq_bin_expected_from_dn/ds_log20','n_seq_bin_expected_from_usage_log20','n_seq_expected_from_dn/ds_log20','n_seq_expected_from_usage_log20','n_seq_diff_dnds_log10','n_seq_diff_dnds_log20','n_seq_diff_usage_log10','n_seq_diff_usage_log20','dnds_mean','dnds_median','dnds_var','dn_mean','dn_median','dn_var','ds_mean','ds_median','ds_var','filt_length','frac_NA','frac_SE_high','frac_dS<0.1','frac_dS>0.8','orig_length', 'dnds_from_usage', 'dnds_from_usage_cons','dnds_from_usage_sites80','dnds_from_usage_sites80_cons','COG','func_gr_all','func_gr','descr','geneID', 'nodes','branch_length_distr','fraction_allowed_subs','gamma','mode','replicate','run','suffix','tree_len_scale','dnds_mean_tree','dnds_median_tree','dnds_var_tree','nonsyn_subs_count','syn_subs_count','total_treelength']
alldf=[vdfr, edfr, gdfr, sdfr, cdfr]
allnames=['vertebrates', 'enterobacterales', 'gammaproteobacteria', 'simulated', 'cogs']
for i in range(5):
    cols=[col for col in keepcol if col in list(alldf[i].columns)]
    findf=alldf[i][cols]
    findf.to_csv(path+allnames[i]+'_summary.csv', index=False)